# Markov State Models
Markov state Models (MSMs) are kinetic models that describe the conformational dynamics of proteins in terms of jumps between the conformational states of the protein. 
A MSMs can be thought as a map of the conformational space a molecule explores and it consists of 

(1) a set of conformational states which are corresponding to the local minima in the free energy landscape

(2) a transition probability matrix which stands for the rate for transitioning across the barrier separating two states.

Recently, MSMs have had success at modeling long-time statistical dynamics.

## building a MSMs

trajectories --> microstates clustering --> macrostates lumping --> validating

1. Microstates clustering of simulation data
2. lumping microstates into macrostates
3. Validating the MSM

In [5]:
# load simulation data
import mdtraj as md

traj = md.load('md.pdb')
print traj
print traj[0]

top = traj.topology

<mdtraj.Trajectory with 101 frames, 4018 atoms, 401 residues, and unitcells>
<mdtraj.Trajectory with 1 frames, 4018 atoms, 401 residues, and unitcells>


In [6]:
trajectories = [traj, traj]
trajectories

[<mdtraj.Trajectory with 101 frames, 4018 atoms, 401 residues, and unitcells at 0x7f183ffa0190>,
 <mdtraj.Trajectory with 101 frames, 4018 atoms, 401 residues, and unitcells at 0x7f183ffa0190>]

## microstates clustering

### kinetically-relevant geometric clustering
* a clustering that only groups conformations together if the system can transition between them quickly relative to transitions between clusters.
* two conformations with a 5 Å RMSD could (1) fall within the same free energy basin if they only differ by pivoting of a hinge motion while (2) another pair of conformations separated by the same distance could fall in different basins if they differ by strand pairings in a beta sheet.
* in fact, two conformations separated by only a few A are likely to inter-convert rapidly.
* measure the average RMSD between every pair of temporally adjacent conformations in each trajectory and to ensure all-heavy-atom 2-2.5 A.

### rich & poor simulation data
* rich case: trajectories from the start conformation to the end conformation.
* poor case: partial or periodical trajectories.

In [11]:
# featurize simulation data
from msmbuilder.featurizer import SuperposeFeaturizer

indices = [atom.index for atom in top.atoms if atom.element.symbol in ['C', 'O', 'N']]
featurizer = SuperposeFeaturizer(indices, trajectories[0][0])
sequences = featurizer.transform(trajectories)

### Estimating the transition probability matrix
#### transition count matrix C(τ)
The transition count matrix C(τ), which denotes the number of observed transitions from state i at time t to state j at time t + τ, where τ is the lag time of the model.
* Markovian in physical sense: next conformation is simply a deterministic function of the system's current state.
* lag time or observation interval τ
* the Chapman-Kolmogorov equation: taking n steps with an MSM with a lag
* time of τ should be equivalent to an MSM with a lag time of nτ.

#### transition probability matrix T(τ)
the transition probability matrix T(τ) includes the probability of transitions from state i to state j in
a certain time interval τ is obtained by normalizing C(τ) with the sum of all transitions from state i. 
* To ensure that the total population of all the states is conserved, a maximum likelihood estimate of the transition probability matrix that obeys the detailed balance is obtained. 
* The eigenvalues/eigenvectors of the transition matrix gives information about the aggregate transitions between subsets of states in the model and what timescales these transitions occur on. 
* The equilibrium populations of the individual states are estimated from the first eigenvector of the transition probability matrix. 
* The timescales of the dynamical processes occurring on the conformational landscape of the protein can be obtained by estimating the eigenvalues of this matrix.

In [26]:
from msmbuilder.cluster import KCenters
from msmbuilder.msm import MarkovStateModel

msmts0, msmts1 = {}, {}
lag_times = [1, 10, 20, 30, 40]
n_states = [4, 8, 16, 32, 64]

for n in n_states:
    msmts0[n] = []
    msmts1[n] = []
    for lag_time in lag_times:
        assignments = KCenters(n_clusters=n).fit_predict(sequences)
        msm = MarkovStateModel(lag_time=lag_time, n_timescales=3, verbose=True).fit(assignments)
        timescales = msm.timescales_
        msmts0[n].append(timescales[0])
        msmts1[n].append(timescales[1])
        print('n_states=%d\tlag_time=%d\ttimescales=%s' % (n, lag_time, timescales[0:2]))
    print()

MSM contains 2 strongly connected components above weight=1.00. Component 0 selected, with population 61.000000%


IndexError: index 1 is out of bounds for axis 0 with size 1

## lumping microstates into macrostates
* macrostates are coarse-graining microstate models.
* kinetically-relevant states require that structures within a state can inter-convert on timescales faster than the lag time. 
* increasing the lag time means that states can get larger and more coarse grained.

### Perron cluster cluster analysis (PCCA)

## Adaptive sampling
In the adaptive sampling scheme, MSMs are built after an initial dataset is obtained to identify new conformational states of protein, which are then used as the starting points for future simulations.

Adaptive sampling procedures seek to identify the starting configurations for future simulations such that the conformational landscapes can be sampled efficiently.

This procedure ensures that the future simulations are started from the edges of the conformational landscape, thereby avoiding the already well-sampled stable states.

### adaptive seeding method (ASM)
### error analysis technique (EAT)
* determine the main contributors to the error which gives rise to the greatest uncertainties.
* each row corresponds to transitions from a single state, so if we find that one row contributes the most to the uncertainty , we can decrease the uncertainty from that row by generating new transitions from that state.

## Validating the MSMs
Testing whether an MSM is self-consistent.
### Swope–Pitera eigenvalue test
### Chapman–Kolmogorov test
### Bayesian model selection approaches.

## future work
1. state scalable MSMs.
2. extendible clustering method and training data.